In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler  

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
Y_train = train_data['target']
X_train = train_data.drop(['target'], axis=1)
final_output = pd.DataFrame()
final_output['id'] = test_data['id']

In [ ]:
def data_preprocessing(data):
    # Converting categorical columns into numerical columns
    le = LabelEncoder()
    for col in data.columns:
        if col.find('cat') != -1:
            data[col] = le.fit_transform(data[col])
    # Data Scaling        
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return data

In [ ]:
X_train = data_preprocessing(X_train)
test_data = data_preprocessing(test_data)

In [ ]:
# define the base model
level0 = list()
level0.append(('cat', CatBoostRegressor()))
level0.append(('xgb', XGBRegressor()))
level0.append(('lgbm', LGBMRegressor()))
level0.append(('SGD', SGDRegressor(max_iter=1000, tol=1e-3)))

level1 = LinearRegression()


In [ ]:
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model.fit(X_train, Y_train)

In [ ]:
model.score(X_train, Y_train)

In [ ]:
target = model.predict(test_data)
final_output['target'] = target

In [ ]:
final_output.head()

In [ ]:
final_output.to_csv("submission.csv", index=False)